### Import statements

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

## Animated Plots ([Docs](https://matplotlib.org/stable/tutorials/introductory/animation_tutorial.html))

### Animation Classes

Matplotlib provides 2 different classes to create animated plots.

1. FuncAnimation: Generate data for first frame and then modify this data for each frame to create an animated plot.

2. ArtistAnimation: Generate a list (iterable) of artists that will be drawn in each frame in the animation.

FuncAnimation is more efficient in terms of speed and memory as it draws an artist once and then modifies it. On the other hand ArtistAnimation is flexible as it allows any iterable of artists to be animated in a sequence.

### FuncAnimation ([Docs](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html#matplotlib.animation.FuncAnimation))

> ``matplotlib.animation.FuncAnimation(fig, func, frames=None, init_func=None, fargs=None, save_count=None, *, cache_frame_data=True, **kwargs)`` is a TimedAnimation subclass that makes an animation by repeatedly calling a function *func*.

<u> Function Parameters</u>

- fig: The figure object onto which the animation is drawn.
- func: The function to call at each frame. The function definition follows a definite pattern e.g. `def func(frame, **func_args)` where frame is the next value in *frames*. It is often more convenient to provide the arguments to the *func* function using `functools.partial()`. In this way it is also possible to pass keyword arguments. To pass a function with both positional and keyword arguments, set all arguments as keyword arguments, just leaving the frame argument unset e.g:

        def func(frame, art, *, y=None):
            ...
        ani = FuncAnimation(fig, partial(func, art=ln, y='foo'))

- frames: Source of data to pass to *func* in each iteration (frame) of the animation. The length of the animation is deduced from this parameter. It is an optional argument and can be any of, iterable, int, generator function, or None.
    - If an iterable, then simply use the values provided.
    - If an integer, then equivalent to passing *range(frames)*
    - If a generator function, then must have the signature, *def gen_function() -> obj*
    - If None, then equivalent to passing *itertools.count*
    In all of these cases, the values in frames is simply passed through to the user-supplied func and thus can be of any type.
- interval: time in milliseconds between drawing of two frames.

**`Note:`** You must store the created Animation in a variable that lives as long as the animation should run. Otherwise, the Animation object will be garbage-collected and the animation stops.

Animating using FuncAnimation would usually follow the following structure:

1. Plot the initial figure, including all the required artists. Save all the artists in variables so that they can be updated later on during the animation.
2. Create an animation function that updates the data in each artist to generate the new frame at each function call. The update function should use the `set_*()` functions for different artists to modify the data. **Tip:** use the plt.setp() to inspect which attributes can be changed.
3. Create a FuncAnimation object with the Figure and the animation function, along with the keyword arguments that determine the animation properties.
4. Use animation.Animation.save or pyplot.show to save or show the animation.

### ArtistAnimation ([Docs](https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.ArtistAnimation.html#matplotlib.animation.ArtistAnimation))

ArtistAnimation can be used to generate animations if there is data stored on various different artists. This list of artists is then converted frame by frame into an animation. For example, when we use Axes.barh to plot a bar-chart, it creates a number of artists for each of the bar and error bars. To update the plot, one would need to update each of the bars from the container individually and redraw them. Instead, `animation.ArtistAnimation` can be used to plot each frame individually and then stitched together to form an animation.

> `matplotlib.animation.ArtistAnimation(fig, artists, *args, **kwargs)` is a TimedAnimation subclass that creates an animation by using a fixed set of Artist objects.

<u>Function Parameters</u>

- fig: The figure object used to get needed events, such as draw or resize.
- artists (list): Each list entry is a collection of Artist objects that are made visible on the corresponding frame. Other artists are made invisible.
- interval (int, default: 200): Delay between frames in milliseconds.
- repeat_delay (int, default: 0): The delay in milliseconds between consecutive animation runs, if repeat is True.
- repeat (default: True): Whether the animation repeats when the sequence of frames is completed.

**`Note:`** Before creating an instance, all plotting should have taken place and the relevant artists saved. You must store the created Animation in a variable that lives as long as the animation should run. Otherwise, the Animation object will be garbage-collected and the animation stops.

> Let's say we want to animate the motion of three projectiles thrown from h1, h2 and h3 height at an angle of theta1, theta2 and theta3 respectively

In [2]:
import matplotlib.animation as animation
from functools import partial

In [3]:
mpl.rcParams["text.usetex"] = True

In [4]:
import math

In [8]:
mpl.rcParams["backend"] = "TkAgg"

# first let's create the figure and axes to draw on
fig, ax = plt.subplots(layout="constrained")

# let's add axes titles, axis titles and other artists that will not be updated
ax.set_title("Visualizing the motion of projectiles")
ax.set_xlabel("Distance travelled in the horizontal direction")
ax.set_ylabel("Distance travelled in the vertical direction")

ax.set_ylim(bottom=0)
ax.set_xlim(left=0)

fig.suptitle(r"$\underline{Projectile\ motion\ animation}$")

# the data
g = 9.81
h = [0, 2, 2.8]
theta = [45, 30, 60]
v0 = [50, 45, 55]
t_upto = [
    math.ceil(2 * v0[i] * math.sin(math.pi * theta[i] / 180) / g)
    for i in range(len(theta))
]

# define how many frames do you want in your animation
no_of_frames = 100


# formulas
def find_x_and_y(h, alpha, u, total_flight_time, g=9.81, no_of_frames=no_of_frames):
    time_range = np.linspace(0, total_flight_time, no_of_frames)
    x = [u * math.cos(math.pi * alpha / 180) * t for t in time_range]
    y = [
        h + u * math.sin(math.pi * alpha / 180) * t - 0.5 * g * (t**2)
        for t in time_range
    ]
    return x, y


# now let's create the artists whose value will change in each frame
x0, y0 = find_x_and_y(h[0], theta[0], v0[0], t_upto[0], g)
# x1, y1 = find_x_and_y(h[1], theta[1], v0[1], t_upto[1], g)
# x2, y2 = find_x_and_y(h[2], theta[2], v0[2], t_upto[2], g)

artists = []


for i in range(no_of_frames):
    line0 = ax.plot(x0[i], y0[i], marker="*")
    # line1 = ax.plot(x1[i], y1[i], marker=">")
    # line2 = ax.plot(x2[i], y2[i], marker="^")
    artists.append(line0)

ani = animation.ArtistAnimation(fig, artists=artists)

plt.show()